In [ ]:
import os
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import DashScopeEmbeddings  
from langchain_core.prompts import ChatPromptTemplate
import pickle


In [ ]:

dashscope_api_key = "<put_your_dashscope_api_key_here>"
dashscope_base_url = "<put_your_dashscope_base_url_here>"


with open("./Data/all_splits.pkl", "rb") as f:
    all_splits = pickle.load(f)

In [3]:

# Build your own system prompt in Arabic or adjusted tone
prompt = ChatPromptTemplate.from_messages([
    ("system",  " أنت عالم فقه مسلم فى غايه الذكاء مقدم لك بيانات لتستمد منها اجاباتك بناء على سؤال مقدم لك واعلم انه يوجد ايات قرءانية مكتوبه بطريقه خطأ لا تستمد منها أيضا تجنب الحروف غير العربية"),
    ("human", "استخدم السياق التالي للإجابة على السؤال:\n\n{context}\n\nالسؤال: {question}\n\nالإجابة:")
])



In [ ]:


embeddings = DashScopeEmbeddings(
    dashscope_api_key=dashscope_api_key,
    model="text-embedding-v1"  
)

# Create vectorstore
vector_store = Chroma(embedding_function=embeddings)
_ = vector_store.add_documents(documents=all_splits)

retriever = vector_store.as_retriever(   
    search_type="similarity",    # or "mmr", "similarity_score_threshold"
    search_kwargs={"k": 3}       # how many results to retrieve
)


In [ ]:
# Configure LLM for DashScope
llm = ChatOpenAI(
    model_name="qwen3-8b", 
    openai_api_key=dashscope_api_key,
    openai_api_base=dashscope_base_url,
    temperature=0,
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


### Run evaluation

In [ ]:
import pandas as pd
from tqdm import tqdm

Data = pd.read_csv(".\evaluation_dataset_plus_164.csv")

In [42]:
model = "deepseek"
Data[model] = ''
for i in tqdm(range(Data.shape[0]), desc="Processing questions"):
# Re-run the specific questions that had errors
    question = Data['question'][i]
    try:
        Data.loc[i, model] = rag_chain.invoke(question)
    except :
        try:
            # Fallback: Try getting response without context
            prompt_without_context = ChatPromptTemplate.from_messages([
                ("system", " أنت عالم فقه مسلم فى غايه الذكاء"),
                ("human", "{question}")
            ])
            chain_without_context = prompt_without_context | llm | StrOutputParser()
            out = chain_without_context.invoke({"question": question})
            Data.loc[i, model] = out
            print(f"Got fallback response for question {i}")
        except Exception as e2:
            print(f"Both attempts failed for question {i}: {e2}")
            Data.loc[i, model] = "No Response"


Processing questions:   1%|          | 1/163 [00:55<2:28:56, 55.16s/it]

Got fallback response for question 0


Processing questions:   3%|▎         | 5/163 [04:40<2:15:03, 51.29s/it]

Got fallback response for question 4


Processing questions:   9%|▉         | 15/163 [11:12<1:32:23, 37.46s/it]

...

Processing questions:  88%|████████▊ | 143/163 [2:34:05<14:48, 44.42s/it]

Got fallback response for question 142


Processing questions:  93%|█████████▎| 152/163 [2:40:31<09:45, 53.27s/it]

Got fallback response for question 151


Processing questions: 100%|██████████| 163/163 [2:47:27<00:00, 61.64s/it]
